# December 2017: Advent of Code

## Common imports & library functions

In [33]:
from collections import defaultdict
import doctest
import heapq
import itertools
import math
import numpy as np

# Cribbed from norvig@
def nth(iterable, n, default=None):
    "Returns the nth item of iterable, or a default value"
    return next(itertools.islice(iterable, n, None), default)

## Day 1: Inverse Captcha

In [ ]:
def solve_captcha(captcha, offset=1):
    """
    >>> solve_captcha('1122')
    3
    >>> solve_captcha('1111')
    4
    >>> solve_captcha('1234')
    0
    >>> solve_captcha('91212129')
    9
    >>> solve_captcha('1212', 2)
    6
    >>> solve_captcha('1221', 2)
    0
    >>> solve_captcha('123425', 3)
    4
    >>> solve_captcha('123123', 3)
    12
    >>> solve_captcha('12131415', 4)
    4
    """
    return sum(int(captcha[i]) for i in range(len(captcha))
               if captcha[i] == captcha[i-offset])

In [ ]:
# Run unit tests
doctest.testmod()

In [ ]:
# Final answer
with open('day1_captcha.txt') as f:
    captcha = f.read().strip()
    print('Part 1: ', solve_captcha(captcha))
    print('Part 2: ', solve_captcha(captcha, len(captcha)//2))

## Day 2: Corruption Checksum

In [ ]:
def minmax_sum(row):
    return max(row) - min(row)

def even_quotient(row):
    for i in range(len(row)):
        for j in range(len(row)):
            if i == j: continue
            if row[i] % row[j] == 0: return row[i] // row[j]

def solve_checksum(spreadsheet, row_checksum=minmax_sum):
    """
    >>> solve_checksum('1 1\\n2 2', minmax_sum)
    0
    >>> solve_checksum('40 41\\n1 0 3 9', minmax_sum)
    10
    >>> solve_checksum('5 1 9 5\\n7 5 3\\n2 4 6 8', minmax_sum)
    18
    >>> solve_checksum('5 5\\n2 2\\n3 3', even_quotient)
    3
    >>> solve_checksum('5 9 2 8\\n9 4 7 3\\n3 8 6 5', even_quotient)
    9
    """
    np_spreadsheet = [[int(c) for c in l.split()] 
                      for l in spreadsheet.splitlines()]
    return sum(row_checksum(row) for row in np_spreadsheet)

In [ ]:
# Run unit tests
doctest.testmod()

In [ ]:
# Final answer
with open('day2.txt') as f:
    spreadsheet = f.read().strip()
    print('Part 1: ', solve_checksum(spreadsheet, row_checksum=minmax_sum))
    print('Part 2: ', solve_checksum(spreadsheet, row_checksum=even_quotient))

### Day 3: Spiral Memory

```
37  36  35  34  33  32  31
38  17  16  15  14  13  30
39  18   5   4   3  12  29
40  19   6   1   2  11  28
41  20   7   8   9  10  27
42  21  22  23  24  25  26
43  44  45  46  47  48  49 ...
```

In [83]:
def spiral_to_ring_size(spiral_coord):
    return math.ceil(math.sqrt(spiral_coord)) | 1

def ring_size_to_ring(ring_size):
    return ring_size // 2 + 1

def spiral_to_ring(spiral_coord):
    """
    Returns the ring that the coordinate appears in, starting with 1 as the center.
    
    >>> spiral_to_ring(1)
    1
    >>> spiral_to_ring(4)
    2
    >>> spiral_to_ring(20)
    3
    >>> spiral_to_ring(49)
    4
    """
    return ring_size_to_ring(spiral_to_ring_size(spiral_coord))

_right = lambda r: ((r-1, i) for i in range(-r+2, r))
_top = lambda r: ((i, r-1) for i in range(r-2, -r, -1))
_left = lambda r: ((-r+1, i) for i in range(r-2, -r, -1))
_bottom = lambda r: ((i, -r+1) for i in range(-r+2, r))
def ring_to_cartesians(ring):
    """
    Returns the cartesian coordinates for cells in the ring in CCW order,
    starting from the cell just above the bottom-right corner.
    
    >>> list(ring_to_cartesians(1))
    [(0, 0)]
    >>> list(ring_to_cartesians(2))
    [(1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0), (-1, -1), (0, -1), (1, -1)]
    >>> list(ring_to_cartesians(3))
    [(2, -1), (2, 0), (2, 1), (2, 2), (1, 2), (0, 2), (-1, 2), (-2, 2), (-2, 1), (-2, 0), (-2, -1), (-2, -2), (-1, -2), (0, -2), (1, -2), (2, -2)]
    """
    if ring == 1:
        return [(0, 0)]
    return itertools.chain(_right(ring), _top(ring), _left(ring), _bottom(ring))

def spiral_to_cartesian(spiral_coord):
    """
    >>> spiral_to_cartesian(1)
    (0, 0)
    >>> spiral_to_cartesian(9)
    (1, -1)
    >>> spiral_to_cartesian(7)
    (-1, -1)
    >>> spiral_to_cartesian(28)
    (3, 0)
    """
    ring_size = spiral_to_ring_size(spiral_coord)
    ring = ring_size_to_ring(ring_size)
    inner_ring_size = max(ring_size-2, 0) ** 2
    ring_coord = spiral_coord - inner_ring_size - 1
    return nth(ring_to_cartesians(ring), ring_coord)

def solve_shortest_distance(spiral_coord):
    """
    >>> solve_shortest_distance(1)
    0
    >>> solve_shortest_distance(12)
    3
    >>> solve_shortest_distance(23)
    2
    >>> solve_shortest_distance(1024)
    31
    """
    x, y = spiral_to_cartesian(spiral_coord)
    return abs(x) + abs(y)

In [84]:
spiral_to_cartesian(1)

(0, 0)

In [85]:
# Run unit tests
doctest.testmod()

TestResults(failed=0, attempted=15)

In [86]:
# Final answer
print('Part 1: ', solve_shortest_distance(368078))

Part 1:  371


In [108]:
# Part 2 will require a total reimplementation...
def generate_spiral():
    ring = 1
    while True:
        for cartesian_coords in ring_to_cartesians(ring):
            yield cartesian_coords
        ring += 1

def solve_shortest_distance2(spiral_coord):
    """
    >>> solve_shortest_distance2(1)
    0
    >>> solve_shortest_distance2(12)
    3
    >>> solve_shortest_distance2(23)
    2
    >>> solve_shortest_distance2(1024)
    31
    """
    (x, y) = nth(generate_spiral(), spiral_coord - 1)
    return abs(x) + abs(y)

In [109]:
doctest.testmod()

**********************************************************************
File "__main__", line 9, in __main__.solve_first_larger_than
Failed example:
    solve_first_larger_than(3)
Exception raised:
    Traceback (most recent call last):
      File "/Users/vtalwar/anaconda3/envs/ml/lib/python3.5/doctest.py", line 1320, in __run
        compileflags, 1), test.globs)
      File "<doctest __main__.solve_first_larger_than[0]>", line 1, in <module>
        solve_first_larger_than(3)
      File "<ipython-input-90-218a354e9667>", line 22, in solve_first_larger_than
        total = sum_neighbors(grid, xy)
      File "<ipython-input-90-218a354e9667>", line 3, in sum_neighbors
        x, y = xy
    TypeError: 'int' object is not iterable
**********************************************************************
File "__main__", line 11, in __main__.solve_first_larger_than
Failed example:
    solve_first_larger_than(12)
Exception raised:
    Traceback (most recent call last):
      File "/Users/vtalwar

TestResults(failed=4, attempted=23)

In [107]:
# Final answer part 1 redux
print('Part 1: ', solve_shortest_distance2(368078))

Part 1:  371


In [112]:
# Now, on to part 2
def sum_neighbors(grid, xy):
    x, y = xy
    deltas = [(-1, -1), (-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1)]
    return sum(grid[(x+dx, y+dy)] for (dx, dy) in deltas)

def solve_first_larger_than(seek_value):
    """
    >>> solve_first_larger_than(3)
    4
    >>> solve_first_larger_than(12)
    23
    >>> solve_first_larger_than(23)
    25
    >>> solve_first_larger_than(500)
    747
    """
    grid = defaultdict(int)
    for xy in generate_spiral():
        total = sum_neighbors(grid, xy) or 1
        if total > seek_value:
            return total
        grid[xy] = total

In [113]:
doctest.testmod()

TestResults(failed=0, attempted=23)

In [114]:
# Final answer part 2
print('Part 2: ', solve_first_larger_than(368078))

Part 2:  369601
